In [3]:
import gensim

# Setup nltk corpora path and Pubmed Word2Vec location
pubmed_vec_file = '/Users/timrand/nltk_data/corpora/PubMed-w2v.bin'

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format(pubmed_vec_file, binary=True)

In [9]:
#given the PubMed-w2v model, we can search for words the most similar to a given word. 
#interestingly, for molecules, the assocaited words are other 
#these are proteins/genes in the pathway. This is because molecule names are rarely used, 
#but when they are they are in association with each other. Therefore, they have high
#probability of being used near each other and nowhere else. 

TP53_assoc = model.most_similar('TP53' ,topn=10)
TP53_assoc

/Users/timrand/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('k-ras', 0.8571987748146057),
 ('CDKN2A', 0.8442618250846863),
 ('KRAS2', 0.8374961614608765),
 ('K-RAS', 0.8317775130271912),
 ('RB1', 0.827904224395752),
 ('Tp53', 0.8247050046920776),
 ('KRAS', 0.8244420289993286),
 ('PIK3CA', 0.82399982213974),
 ('K-ras', 0.8214455842971802),
 ('CTNNB1', 0.8166534900665283)]

In [13]:
import pickle
nomenclature_approved = pickle.load( open( "pickles/nomenclature_approved.pkl", "rb" ) )


In [32]:
#this should be changed to a dictionary for easier lookup
nom_dict = {}
for i,gene in enumerate(nomenclature_approved):
    try:
        nom_dict[gene[0]] = gene[1]
    except:
        print(f'failed on {gene} {i}')

#pickle.dump( nom_dict, open( "nomenclature_approved_dict.pkl", "wb" ) )
print(nom_dict['A2ML1-AS2'])


failed on {} 56274
1


In [40]:
nomen = pickle.load( open( "nomenclature_approved_dict.pkl", "rb" ) )

In [76]:

    
def bin_approve_fail_notFound(genes):
    """
    bin_fail_notFound() takes a gene string or list of genes and returns the names
    binned into lists in the following order: approve, failed, not found.
    """
    nomen_dict = pickle.load( open( "nomenclature_approved_dict.pkl", "rb" ) )
    approved = []
    failed = []
    no_entry = []
    if type(genes) == str:
        genes = [genes]
    
    for gene in genes:
        gene = gene.upper()
        try:
            approval_status = nomen_dict[gene]
            if approval_status == 1:
                approved.append(gene)
            else:
                failed.append(gene)
        except:
            no_entry.append(gene)
    return (approved, failed, no_entry)
 

    

    

In [77]:
bin_approve_fail_notFound(['AGO1', 'TP53', 'CDKN1a', "A-GAMMA3'E",'RB', 'junkynono'])

(['AGO1', 'TP53', 'CDKN1A'], ["A-GAMMA3'E"], ['RB', 'JUNKYNONO'])